In [33]:
import pandas as pd

# Part 1. Scrapeing Wikipedia

The wanted table is the first entry in the list

In [34]:
postal_codes = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Cleaning

In [35]:
postal_codes = postal_codes[postal_codes['Borough'] != 'Not assigned'].reset_index(drop=True)
postal_codes.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [36]:
postal_codes.shape

(103, 3)

# Part 2. Adding the coordinates

In [37]:
coords = pd.read_csv('Geospatial_Coordinates.csv')

In [38]:
neighborhoods = pd.merge(postal_codes, coords, left_on='Postal Code', right_on='Postal Code', how='inner')

In [39]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Limit to boroughs with Toronto in the name

In [40]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]

In [41]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


# 3. Plotting and clustering

In [42]:
from geopy.geocoders import Nominatim 
import requests
from pandas.io.json import json_normalize
import folium
pd.options.display.max_columns = None
pd.options.display.max_rows = None



Get coordinates for folium center

In [43]:
address = 'Toronto, CA'

location = Nominatim(user_agent="toronto").geocode(address)
latitude = location.latitude
longitude = location.longitude

Plot the neighborhoods to see whats in the set

In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Start foursquare querys

In [65]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Using the function from the lab

In [46]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Queried once and wrote the result to a file to avoid unnecesary queries

In [67]:
#toronto_venues = getNearbyVenues(neighborhoods['Neighbourhood'], neighborhoods['Latitude'], neighborhoods['Longitude'])
#toronto_venues.to_csv('toronto_venues.csv')
toronto_venues = pd.read_csv('toronto_venues.csv')

In [68]:
toronto_venues.head()

,Unnamed: 0,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,4,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


There are venues with a category 'Neighborhood', renameing to avoid conflict with 'Neighborhood' label

In [69]:
toronto_venues.loc[toronto_venues['Venue Category'] == 'Neighborhood','Venue Category'] = 'Neighborhood Venue'

One hot encoding of venue categories

In [70]:
toronto_onehot = pd.get_dummies(toronto_venues['Venue Category'])

Check shapes

In [71]:
toronto_venues.shape

(1624, 8)

In [72]:
toronto_onehot.shape

(1624, 237)

Add 'Neighborhood'

In [73]:
toronto_onehot = pd.concat([toronto_venues['Neighborhood'],toronto_onehot],axis=1)

In [75]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean()

In [76]:
toronto_grouped.shape

(39, 237)

In [77]:
for neigh in toronto_grouped.index:
    print(f"{neigh}:")
    print(toronto_grouped.loc[neigh].sort_values(ascending=False)[:5])
    print('\n')

Berczy Park:
Coffee Shop           0.090909
Cocktail Bar          0.036364
Seafood Restaurant    0.036364
Bakery                0.036364
Beer Bar              0.036364
Name: Berczy Park, dtype: float64


Brockton, Parkdale Village, Exhibition Place:
Café              0.130435
Nightclub         0.086957
Coffee Shop       0.086957
Breakfast Spot    0.086957
Grocery Store     0.043478
Name: Brockton, Parkdale Village, Exhibition Place, dtype: float64


Business reply mail Processing Centre, South Central Letter Processing Plant Toronto:
Gym / Fitness Center    0.0625
Auto Workshop           0.0625
Comic Shop              0.0625
Park                    0.0625
Pizza Place             0.0625
Name: Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, dtype: float64


CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport:
Airport Lounge     0.1250
Airport Service    0.1250
Boutique           0.0625
Plane     

# Clustering

### I'm keeping the full frequency data and don't reduce it to the most frequent venues

In [78]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

Do the clustering

In [113]:
k = 5
kmeans = KMeans(n_clusters=k, random_state=1).fit(toronto_grouped)

Generate df with the labels and neighborhood infos

In [110]:
toronto_grouped_labeled = toronto_grouped.copy()

toronto_grouped_labeled['Cluster Label'] = kmeans.labels_

toronto_clusters = toronto_grouped_labeled.reset_index().join(neighborhoods.set_index('Neighbourhood'), on='Neighborhood')

toronto_clusters.head()

Generate map with the labels

In [111]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_clusters['Latitude'], toronto_clusters['Longitude'], toronto_clusters['Neighborhood'], toronto_clusters['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [112]:
for neigh in toronto_clusters.loc[toronto_clusters['Cluster Label'] == 0,'Neighborhood']:
    print(neigh)
    print(toronto_grouped.loc[neigh].sort_values(ascending=False)[:5])
    print('\n')

Berczy Park
Coffee Shop           0.090909
Cocktail Bar          0.036364
Seafood Restaurant    0.036364
Bakery                0.036364
Beer Bar              0.036364
Name: Berczy Park, dtype: float64


Brockton, Parkdale Village, Exhibition Place
Café              0.130435
Nightclub         0.086957
Coffee Shop       0.086957
Breakfast Spot    0.086957
Grocery Store     0.043478
Name: Brockton, Parkdale Village, Exhibition Place, dtype: float64


Business reply mail Processing Centre, South Central Letter Processing Plant Toronto
Gym / Fitness Center    0.0625
Auto Workshop           0.0625
Comic Shop              0.0625
Park                    0.0625
Pizza Place             0.0625
Name: Business reply mail Processing Centre, South Central Letter Processing Plant Toronto, dtype: float64


CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Airport Lounge     0.1250
Airport Service    0.1250
Boutique           0.0625
Plane         